In [1]:
import pandas as pd
import praw
import json

with open('.reddit.json') as f:
    config = json.load(f)
    reddit = praw.Reddit(**config)
    subreddit = reddit.subreddit('GPTStore')
    posts = pd.DataFrame()
    for submission in subreddit.new(limit = None):
        post = {
            'Title': submission.title,
            'Question': submission.selftext,
            'subreddit': submission.subreddit.display_name,
            'Link': "https://www.reddit.com" + submission.permalink,
            'Creation Time': submission.created,
            'Reply Count': submission.num_comments,
            'Upvote Ratio': submission.upvote_ratio
        }
        post = pd.DataFrame([post])
        posts = pd.concat([posts, post], ignore_index=True)
    posts.to_json('reddit_store_posts.json', orient='records', indent=4)


/Users/jimmy/Documents/GitHub/GPT-Store-REsearch/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import numpy as np
import requests

tag_store_links = []

driver = webdriver.Chrome()
driver.implicitly_wait(5)

url = 'https://community.openai.com/tags'
driver.get(url)

for tag in driver.find_elements(By.XPATH, '//div[@class="tag-box"]/a'):
    tag_name = tag.get_attribute('data-tag-name')
    if 'store' in tag_name.lower():
        tag_store_links.append(tag.get_attribute('href'))
        
post_store_links = set()

for tag_link in tag_store_links:
    driver.get(tag_link)
    for post in driver.find_elements(By.XPATH, '//tbody[@class="topic-list-body"]/tr'):
        post_link = post.find_element(By.XPATH, './/a[@role="heading"]').get_attribute('href')
        post_store_links.add(post_link)
        
posts = pd.DataFrame()
for post_link in post_store_links:
    json_data = requests.get(post_link + '.json').json()
    post = {}
    post['Title'] = json_data['title']
    post['Creation Time'] = json_data['created_at']
    post['View Count'] = json_data['views']
    post['Reply Count'] = json_data['reply_count']
    post['Like Count'] = json_data['like_count']
    post['Link'] = post_link
    post['Tags'] = json_data['tags']
    post['Question'] = json_data['post_stream']['posts'][0]['cooked']
    accpeted_answer = np.nan
    for reply in json_data['post_stream']['posts'][1:]:
        if reply['accepted_answer']:
            accpeted_answer = reply['cooked']
            break
    post['Accepted Answer'] = accpeted_answer
    post = pd.DataFrame([post])
    posts = pd.concat([posts, post], ignore_index=True)
posts.to_json('openai_store_posts.json', orient='records', indent=4)

driver.quit()


In [ ]:
import pandas as pd
import webbrowser

def input_challenge_type(link):
    print(link)
    
    # Adjusted mapping to handle empty string as 'na'
    choice_mapping = {
        "K": 'knowledge',
        "": 'na',  # Using empty string to represent 'N'/'na' choice
        "P": 'problem',
    }
    valid_options = ["P", "K", ""]  # Empty string as a valid option
    
    webbrowser.open(link)
    choice = input("Choose an option - 'P', 'K', or press Enter for 'N': ").upper()

    # Adjusting the condition to check for an empty string as well
    while choice not in valid_options:
        print("Invalid choice. Please choose 'P', 'K', or press Enter for 'N' only.")
        choice = input("Choose an option - 'P', 'K', or press Enter for 'N': ").upper()

    return choice_mapping[choice]

post_files = ['reddit_plugin_posts.json']
for post_file in post_files:
    df = pd.read_json(post_file)
    for index, row in df.iterrows():
        df.at[index, 'Challenge Type'] = input_challenge_type(row['Link'])
        if index % 5 == 0:
            df.to_json(post_file, orient='records', indent=4)
    df.to_json(post_file, orient='records', indent=4)